<a href="https://colab.research.google.com/github/izlamastfgil/Learn-LLM/blob/main/RAG_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain faiss-cpu pypdf langchain_community sentence-transformers transformers langchain-huggingface gradio

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load your rules PDF
loader = PyPDFLoader("legal.pdf")  # Upload to Colab first
documents = loader.load()

# Chunk the text
splitter = RecursiveCharacterTextSplitter(chunk_size=250, chunk_overlap=50)
docs = splitter.split_documents(documents)

print(f"Total chunks: {len(docs)}")

Total chunks: 1574


In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

# Use a lightweight embedding model
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from langchain_community.vectorstores import FAISS

# Build FAISS index
db = FAISS.from_documents(docs, embeddings)
retriever = db.as_retriever(search_kwargs={"k": 2})


In [ ]:
from transformers import pipeline
from langchain_huggingface import HuggingFacePipeline

# Load a small LLM for QA
pipe = pipeline("text2text-generation", model="google/flan-t5-xl", device=0, torch_dtype="auto",max_length=512)

llm = HuggingFacePipeline(pipeline=pipe)


config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.45G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

template = """
You are a strict legal assistant. Answer in **two parts**.

Part 1: Quote the relevant text from the Rules exactly as it appears.
Part 2: Explain in simple terms what this rule means and how it applies.

If the answer is not in the Rules, respond exactly: "Not found in rules."
Do not invent information.

Rules:
{context}

Question: {question}

Answer:
"""
prompt = PromptTemplate(
    template=template,
    input_variables=["context", "question"]
)


qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True
)

In [ ]:
query = "What is limited partnership?"
result = qa.invoke({"query": query})

print("Answer:", result["result"])
for doc in result["source_documents"]:
    print(f"- Page {doc.metadata.get('page_label')}")


Answer: A limited partnership is very similar to a regular partnership. There are a few "key" differences. A. Structure . First the limited partnership has limited partners and at least "one" general partner.
- Page 123
- Page 123


In [ ]:
import gradio as gr

def chat(query):
    result = qa({"query": query})
    return result["result"]

In [ ]:
demo = gr.Interface(fn=chat, inputs="text", outputs="text", title="Rules RAG Chatbot")
demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d9e6a00f640e7f4143.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
